# Get the most ordered custom-products

## Parameters

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Catalog/Custom_products/data'
FILE_NAME = 'custom_products.csv'
STOPWORDS = [
    'heb','empty','si','no','ni','sin','que','q','un','una','uno','el','la','los',
    'en','de','y','mi','para','por','favor','porfavor','porfa','has','re','esten',
    'muy','ma','lo','se','sea','solo','este','do','con','hay','gracia','pieza','gr',
    'gramo','kg','kilo','marca','bolsa','caja','paquete','grande','sabor','color',
    'rojo','blanco','traer']

## Code

In [2]:
from pathlib import Path

from numpy import nan
from emoji import demojize
from re import sub, UNICODE
from unicodedata import normalize
from pandas import DataFrame, read_csv

from sklearn.feature_extraction.text import CountVectorizer

class CustomProducts:
    def __init__(self, base_dir: str, file_name: str, stopw: list) -> None:
        '''
        Inicializa la clase recibiendo un directorio un nombre de archivo
        '''
        # Obtiene un directorio como texto y convertirlo a tipo Path para unir directorios, buscar archivos, etc.
        self.base_dir = Path(base_dir)
        # Guarda el nombre del archivo como atributo
        self.file_name = file_name
        # Guarda la lista de stopwords para omitir
        self.stopwords = stopw

        # Concatena el directorio y el nombre del archivo
        self.file_path = self.base_dir.joinpath(self.file_name)
        # Revisa si el archivo existe
        if not self.file_path.is_file(): print(f'It should be a file called "{self.file_name}" at\n{self.base_dir}\n\nBut there is not, add it and try again')


    def clean_text(self, text: str, rem_stop: list, pattern: str="[^a-zA-Z\s]", lower: bool=True, emoji: bool=True, to_singular: bool=True) -> str: 
        '''
        Limpieza de texto
        '''
        # "Traduce" emojis, ej: 🇲🇽 --> :Mexico:
        if emoji: text = demojize(text)

        # Reemplazar acentos: áàäâã --> a
        clean = normalize('NFD', str(text).replace('\n', ' \n ')).encode('ascii', 'ignore')
        # Omitir caracteres especiales !"#$%&/()=...
        clean = sub(pattern, ' ', clean.decode('utf-8'), flags=UNICODE)

        # Mantener sólo un espacio
        clean = sub(r'\s{2,}', ' ', clean.strip())

        # Minúsculas si el parámetro lo indica
        if lower: clean = clean.lower()
        # Omitir la última "s" si el parámetro lo indica
        if to_singular: clean = sub('s\s',' ',clean+' ')

        # Omitir las stopwords indicadas
        clean = ' '.join([x for x in clean.split() if x not in rem_stop])

        # Si el registro estaba vacío, indicar nulo
        if clean in ('','nan'): clean = nan
        return clean

    def data_wrangling(self, col: str='description', top_n=100, **kwargs) -> DataFrame:
        # Obtiene el csv
        df = read_csv(self.file_path)
        df[f'{col}_clean'] = df[col].map(lambda x: self.clean_text(x, rem_stop=self.stopwords)).fillna('')
        cv = CountVectorizer(max_features=top_n, **kwargs)
        cv_fit = cv.fit_transform(df[f'{col}_clean'])
        top = dict(zip(cv.get_feature_names(), cv_fit.toarray().sum(axis=0)))
        top = DataFrame(top, index=['word_count']).T.sort_values('word_count', ascending=False)
        return top

cp = CustomProducts(BASE_DIR, FILE_NAME, STOPWORDS)

## Import

In [3]:
top_n = cp.data_wrangling(top_n=30, ngram_range=(1,3))
top_n

,word_count
queso,44
pan,37
pavo,24
natural,20
rebanada,20
pechuga,20
chocolate,17
pechuga pavo,17
bote,16
crema,15
